In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
import math
from matplotlib.patches import Rectangle
from matplotlib import rcParams

In [ ]:
# import data here 


In [ ]:
def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = float(pearsonr(df[r], df[c])[1])
    return pvalues

In [ ]:
pdf = calculate_pvalues(df).apply(pd.to_numeric)
xsig = []
ysig = []
ssig = []
for c in range(len(pdf.columns)):
    for r in range(len(pdf)):
        pv = pdf.iloc[c,r]
        if r > c and pv < 0.05:
            xsig.append(c+0.5)
            ysig.append(r+0.55)
            if pv < 0.001:
                ssig.append('\u2217')
            elif pv < 0.01:
                ssig.append('+')
            else:
                ssig.append('\u00b7')

In [ ]:
fig, ax = plt.subplots(figsize=(15,12))
fontsize = len(df.columns)*.6
corr = df.corr()

# heatmap mask and color
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
cmap = sns.cm.vlag_r

# plots
heatmap = sns.heatmap(corr, cmap=cmap, cbar_kws=dict(shrink=0.5, location="right"), vmin = -1, vmax = 1, mask=mask,linewidths=1.5)
for i, pdot in enumerate(ssig):
    ax.text(xsig[i],ysig[i],pdot,horizontalalignment='center', verticalalignment='center', fontsize=fontsize, fontfamily='sans-serif')

# legends and title
props = dict(boxstyle='round',facecolor='none', edgecolor='gray')
plt.text(len(pdf.columns)+1, 3, "   p-value \n \u2217 <  0.001  \n + <  0.01 \n \u00b7 <  0.05", bbox=props, fontsize=fontsize, fontfamily='monospace')
# handles, labels = dot.legend_elements(prop="sizes", alpha=0.6, func= lambda x: 0.05-(x/1000))
# legend2 = ax.legend(handles, labels, loc="upper right", title="p-value")
plt.title("Feature Correlation - Download Prediction", size=20)